In [1]:
import sys  
reload(sys)  
sys.setdefaultencoding('utf-8')

In [2]:
import pandas as pd
import json
import numpy as np

In [3]:
#!/usr/bin/env python
#-*-coding=gbk-*-
 
"""
     原始数据，用于建立模型
"""
#缩水版的courses，实际数据的格式应该为 课程名\t课程简介\t课程详情，并已去除html等干扰因素
courses = [           
            u'Writing II: Rhetorical Composing',
            u'Genetics and Society: A Course for Educators',
            u'General Game Playing',
            u'Genes and the Human Condition (From Behavior to Biotechnology)',
            u'A Brief History of Humankind',
            u'New Models of Business in Society',
            u'Analyse Numrique pour Ingnieurs',
            u'Evolution: A Course for Educators',
            u'Coding the Matrix: Linear Algebra through Computer Science Applications',
            u'The Dynamic Earth: A Course for Educators',
            u'Tiny Wings\tYou have always dreamed of flying - but your wings are tiny. Luckily the world is full of beautiful hills. Use the hills as jumps - slide down, flap your wings and fly! At least for a moment - until this annoying gravity brings you back down to earth. But the next hill is waiting for you already. Watch out for the night and fly as fast as you can. ',
            u'Angry Birds Free',
            u'没有\它很相似',
            u'没有\t它很相似',
            u'没有\t他很相似',
            u'没有\t他不很相似',
            u'没有',
            u'可以没有',
            u'也没有',
            u'有没有也不管',
            u'Angry Birds Stella',
            u'Flappy Wings - FREE\tFly into freedom!A parody of the #1 smash hit game!',
            u'没有一个',
            u'没有一个2',
           ]
 


In [36]:
#df_no_dup = pd.read_csv('../data/classifier/bosonNLP/bosonNLP-classifier-tuned.csv')
df = pd.read_csv('/Users/nhu2000/Desktop/huizhou/data/huiwenhua/huizhou_web_content_slim_v2-0.csv')

In [5]:
df_final = df[df['domain'] != 'wuyuan168.com']
df_final = df_final.dropna(subset =['key_words'])
df_final = df_final[df_final['target'] == 1]

In [40]:
docs = df['combined'].str.split()

In [39]:
df_final.info()

In [7]:
df_final.head()

,url,domain,title,segment,key_words,summary,body,target,combined
1,www.hzwh.com/article.asp?id=1316,hzwh.com,用心灵感悟徽文化的博大精深,用 心灵 感悟 文化 徽文化 的 博大 精深 博大精深,文化 文脉 古村落 万善 古民居 可视性 亭榭 统一性 儒家思想 无处不在 昭示 明清两代 ...,宋代兴起的理学是儒家思想的一个流派，明清两代达到巅峰，与中国几千年的文化具有统一性，又呈现多...,徽州自古人文荟萃，丰厚的人文遗产，几乎涵盖了哲学、社会、教育、艺术、宗教、建筑、医...,1,用 心灵 感悟 文化 徽文化 博大 精深 博大精深文化 文脉 古村落 万善 古民居 可视性 ...
2,www.hzwh.com/article.asp?id=1203,hzwh.com,清末民初徽商兴办实业教育,清末 民初 徽商 兴办 实业 教育,徽州 绩溪 创办 茶商 蚕桑 学堂 农业 朱里 邹等 戴琴泉 吴俊德 六县 周协恭 传技 汪...,清光绪末年，休宁茶商戴琴泉等在屯溪朱里成立农业公司，栽桑养蚕，同时创办了初等农业学堂，培养蚕...,清末，称雄商界几百年的徽商，已是日薄西山，渐呈衰败。当时徽商中的部分有识之士已经看到...,1,清末 民初 徽商 兴办 实业 教育徽州 绩溪 创办 茶商 蚕桑 学堂 农业 朱里 邹等 戴琴...
3,www.hzwh.com/article.asp?id=907,hzwh.com,新安画派的绘画风格,新安 画派 的 绘画 风格,新安 画派 山水 画家 创作 观察 学养 篆刻家 范本 熏陶 徽州 灵感 大好 汲取 素材 ...,新安大好山水为当地画家提供了很好的范本，更是他们从中汲取创作灵感的源泉 新安画派注重对自然的...,一、绘画技法上崇尚元四家，推倪瓒为宗师。构图上讲求疏密相衬、山水相生，动与静、远与近...,1,新安 画派 绘画 风格新安 画派 山水 画家 创作 观察 学养 篆刻家 范本 熏陶 徽州 灵...
4,http://www.xinhs.cn/article.asp?id=38700,xinhs.cn,安徽大学研究生在茗洲村开展徽州人生仪礼课题调查活动,安徽 大学 安徽大学 研究 研究生 在 茗 洲 村 开展 徽州 人生 仪礼 课题 调查 活动,徽州 村落 礼仪 了解 习俗 婚丧 吴氏家典 学生 吴氏 婚庆 礼教 文化 座谈 仪礼 民居...,实地考察中，民俗专业的学生与村中 多岁的老人座谈，详细地了解了这个古老村落的婚庆、丧葬习俗...,实地考察中，民俗专业的学生与村中80多岁的老人座谈，详细地了解了这个古老村落...,1,安徽 大学 安徽大学 研究 研究生 茗 洲 村 开展 徽州 人生 仪礼 课题 调查 活动徽州...
5,http://www.xinhs.cn/article.asp?id=80495,xinhs.cn,新安地理练江上的古歙三桥,新安 地理 练江 上 的 古 歙 三桥,太平桥 练江 渔梁坝 三桥 紫阳 浮桥 横跨 惊涛拍岸 冲坏 卧波 刘炳 石拱桥 端平 庆丰...,其中著名的要算建于明朝的太平桥、万年桥和紫阳桥了，三座桥又合称古歙三桥，如长虹卧波一样，横跨...,春天的新安江，美丽如画。站在岸边，我们不住地轻声而叹。歙县县城一带的新安江，...,1,新安 地理 练江 古 歙 三桥太平桥 练江 渔梁坝 三桥 紫阳 浮桥 横跨 惊涛拍岸 冲坏 ...


In [41]:
courses = docs

In [42]:
courses_name = courses

In [43]:
"""
    预处理(easy_install nltk)
"""
def pre_process_cn(courses, low_freq_filter = True):
    """
     简化的 中文+英文 预处理
        1.去掉停用词
        2.去掉标点符号
        3.处理为词干
        4.去掉低频词
 
    """
    import nltk
    import jieba.analyse
    from nltk.tokenize import word_tokenize
    
    texts_tokenized = []
    for document in courses:
        texts_tokenized_tmp = []
        for word in word_tokenize(document):
            texts_tokenized_tmp += jieba.analyse.extract_tags(word,10)
        texts_tokenized.append(texts_tokenized_tmp)   
    
    texts_filtered_stopwords = texts_tokenized
 
    #去除标点符号
    english_punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']
    texts_filtered = [[word for word in document if not word in english_punctuations] for document in texts_filtered_stopwords]
 
    #词干化
    from nltk.stem.lancaster import LancasterStemmer
    st = LancasterStemmer()
    texts_stemmed = [[st.stem(word) for word in docment] for docment in texts_filtered]
    
    #去除过低频词
    if low_freq_filter:
        all_stems = sum(texts_stemmed, [])
        stems_once = set(stem for stem in set(all_stems) if all_stems.count(stem) == 1)
        texts = [[stem for stem in text if stem not in stems_once] for text in texts_stemmed]
    else:
        texts = texts_stemmed
    return texts

In [14]:
lib_texts = pre_process_cn(courses)

TypeError: expected string or buffer

In [11]:
 
"""
    引入gensim，正式开始处理(easy_install gensim)
"""
 
def train_by_lsi(lib_texts):
    """
        通过LSI模型的训练
    """
    from gensim import corpora, models, similarities
 
    #为了能看到过程日志
    #import logging
    #logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
    dictionary = corpora.Dictionary(lib_texts)
    corpus = [dictionary.doc2bow(text) for text in lib_texts]     #doc2bow(): 将collection words 转为词袋，用两元组(word_id, word_frequency)表示
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
 
    #拍脑袋的：训练topic数量为10的LSI模型
    lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=20)
    index = similarities.MatrixSimilarity(lsi[corpus])     # index 是 gensim.similarities.docsim.MatrixSimilarity 实例
    
    return (index, dictionary, lsi)


In [44]:
 
    
    
#库建立完成 -- 这部分可能数据很大，可以预先处理好，存储起来
#(index,dictionary,lsi) = train_by_lsi(lib_texts)
(index,dictionary,lsi) = train_by_lsi(docs)


In [45]:
import cPickle as pickle
fname1 = '../pkl/lsi_model.pkl'
fname2 = '../pkl/lsi_index.pkl'
fname3 = '../pkl/lsi_dictionary.pkl'
with open(fname1, 'w') as f:
    pickle.dump(lsi, f)
    
with open(fname2, 'w') as f:
    pickle.dump(index, f)

with open(fname3, 'w') as f:
    pickle.dump(dictionary, f)

In [26]:
with open(fname1) as f:
    lsi = pickle.load(f)

with open(fname2) as f:
    index = pickle.load(f)

with open(fname3) as f:
    dictionary = pickle.load(f)

In [33]:
print len(index), len(dictionary), len(ml_lsi)

In [27]:
print index

In [28]:
print dictionary

In [13]:
    
    
#要处理的对象登场
target_courses = u'徽文化'
target_text = pre_process_cn(target_courses, low_freq_filter=False)
 


In [57]:
from gensim import corpora, models, similarities
from gensim.similarities import MatrixSimilarity

In [48]:
 
"""
对具体对象相似度匹配
"""
 
#选择一个基准数据
#ml_course = df_final.iloc[3703]['segment'].split()
ml_course = ['徽文化']

 
#词袋处理
ml_bow = dictionary.doc2bow(ml_course)  
 
#在上面选择的模型数据 lsi 中，计算其他数据与其的相似度
ml_lsi = lsi[ml_bow]     #ml_lsi 形式如 (topic_id, topic_value)

print ml_lsi

sims = index[ml_lsi]     #sims 是最终结果了， index[xxx] 调用内置方法 __getitem__() 来计算ml_lsi
 
#排序，为输出方便
sort_sims = sorted(enumerate(sims), key=lambda item: -item[1])

tot = 5

labels, sim_values = zip(*sort_sims[0:tot])

print 'reference doc ---', labels[0], df_final.iloc[labels[0]]['title']

for i in range(1, tot):
    print labels[i], df_final.iloc[labels[i]]['title']
    print labels[i], df_final.iloc[labels[i]]['url']




#查看结果
#print sort_sims[0:10]   #看下前10个最相似的，第一个是基准数据自身
#print courses_name[sort_sims[1][0]]   #看下实际最相似的数据叫什么
#print courses_name[sort_sims[2][0]]   #看下实际最相似的数据叫什么
#print courses_name[sort_sims[3][0]]   #看下实际最相似的数据叫什么



In [54]:
from gensim import corpora, models, similarities
ret = models.LsiModel.get_similarities(ml_lsi)

AttributeError: type object 'LsiModel' has no attribute 'get_similarities'

In [31]:
print ml_lsi

In [49]:
print index

In [107]:
print index

MatrixSimilarity<4979 docs, 20 features>


In [122]:
ret = lsi.show_topics(20, 20)

In [123]:
import re
match = '\"(.*?)\"'
for i in range(0, 20):
    t = ret[i][1]
    s = re.findall('"([^"]*)"', t)
    #print t
    print i, ' '.join(s)


0 文化 吸附 身处 手艺人 手艺 道理 感受 六年 清光绪 嘉定 自宋 文武 徽州 状元 十年 休宁 徽派 保护 徽文化 建筑
1 文化 道理 六年 吸附 手艺 身处 清光绪 嘉定 文武 手艺人 自宋 十年 感受 状元 保护 徽州 徽文化 徽派 黄山 建筑
2 建筑 徽派 古建筑 民居 研究 雕刻 文化 古民居 古建 木雕 徽学 艺术 古村落 砖雕 保护 学术 装饰 徽商 新安 西递
3 保护 徽派 徽商 新安 旅游 古村落 生态 古民居 黄山 利用 资料 艺术 黄山市 网上 网上家园 古建筑 本站 若有 管理员 侵权
4 文化 研究 保护 徽学 歙县 本站 若有 管理员 侵权 网上 来源于 网上家园 资料 网络 徽商 绩溪 休宁 生态 联系 黟县
5 徽学 研究 建筑 若有 管理员 本站 侵权 来源于 明清 网络 网上 网上家园 社会 徽商 联系 文书 安徽 未经许可 资料 随意
6 徽商 协会会员 桐城人 年生 报业 甘臻 研究生 现为 研究 徽学 安徽省 新安 木雕 大会 雕刻 七章 第七章 经营 旅游 江淮
7 徽商 协会会员 桐城人 年生 甘臻 报业 研究生 现为 木雕 黟县 第七章 江淮 西递 新安 安徽省 七章 第七 艺术 雕刻 明清
8 徽商 休宁 新安 徽文化 休宁县 木雕 婺源 雕刻 黟县 故园 传承 屯溪 大会 文化 古村落 徽剧 医学 西递 歙县 祁门
9 新安 绩溪 医学 画派 故园 木雕 胡适 徽商 活动 徽州 民俗 雕刻 周年 汪华 汪公 休宁 学术 保护 汪王 绩溪县
10 绩溪 休宁 故园 黄山 祁门 万安 保护 休宁县 徽墨 旅游 生态 徽派 徽文化 公告 文化遗产 制作 罗盘 国家级 绩溪县 大会
11 绩溪 徽菜 徽剧 黄山 休宁 木雕 休宁县 屯溪 雕刻 黄山市 歙县 万安 新安 徽文化 老街 古村落 罗盘 胡适 民俗 徽班
12 黟县 屯溪 老街 西递 旅游 祁门 宏村 建筑 胡适 徽商 绩溪 故园 保护 木雕 古建筑 学术 周年 祁门县 徽剧 屯溪区
13 徽剧 绩溪 徽班 徽菜 徽文化 京剧 徽派 旅游 歙县 故园 黄山 休宁 文化遗产 研究 胡适 四喜 西递 建筑 戏曲 进京
14 徽剧 新安 保护 徽商 徽班 中国 徽学 故园 生态 屯溪 画派 历史 老街 黄山 文书 研究 医学 京剧 休宁县 徽菜
15 歙县 